# Graded Assignment 2 - Air Quality and Weather in the Netherlands


# Introduction to This Template Notebook

* This is a **group** notebook.
* Make sure you work in a **copy** of `...-template.ipynb`,
**renamed** to `...-yourIDnrs.ipynb`,
where `yourIDnrs` is the TU/e identification numbers of the members of the group.

<div class="alert alert-danger" role="danger">
<h3>Integrity</h3>
<ul>
    <li>In this course you must act according to the rules of the TU/e code of scientific conduct.</li>
    <li>This exercise or graded assignment is to be executed by the members of the group independently from other people.</li>
    <li>You must not copy from the Internet, your friends, books... If you represent other people's work as your own, then that constitutes fraud and will be reported to the Examination Committee.</li>
    <li>Making your work available to others (complicity) also constitutes fraud.</li>
</ul>
</div>

You are expected to work with Python code and Markdown in this notebook.

Proceed in this notebook as follows:
* **Read** the assignment (separate PDF).
* **Write** your decisions/solutions/interpretations in the appropriate sections.
  * For this you can use both Code and Markdown cells. Information about how to use these cells is available in the _Getting Started_ assignment.
* **Run** _all_ code cells (also the ones _without_ your code),
    _in linear order_ from the first code cell.

**Personalize your notebook**:
1. Copy the following line of code:

  ```python
  AUTHOR_ID_NRS = ['1234567', '2234567', '3234567', '4234567']
  ```
1. Paste them between the marker lines in the next code cell.
1. Fill in the _identification numbers_ of all members of the group as a list of strings between the `Author` markers.
1. Run the code cell by putting the cursor there and typing **Control-Enter**.


In [4]:
#// BEGIN_TODO [Author] Name, Id.nr., Date, as strings (1 point)

AUTHOR_ID_NRS = ['2027453', '1989596', '2007630', '2056313']

#// END_TODO [Author]

## Table of Contents

- [Preparation](#Preparation)
    - [Load the libraries](#Load-the-libraries)
- [Part 1a. Hypothesis selection](#Part-1a:-Hypothesis-selection)
- [Part 1b. Hypothesis refinement](#Part-1b:-Hypothesis-refinement)
- [Part 2. Queries and data cleaning](#Part-2:-Queries-and-data-cleaning)
- [Part 3. Hypothesis testing and interpretation](#Part-3.-Hypothesis-testing-and-interpretation)
- [Part 4. Pitching results](#Part-4.-Pitching-results)

## Preparation
### Load the libraries

In [5]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression         # for linear regression
from sklearn.cluster import KMeans                        # for clustering
from sklearn.tree import DecisionTreeClassifier           # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix
from sklearn.model_selection import train_test_split
from treeviz import tree_print                            # to print decision tree (gives an error for now)

import scipy.stats as stats                               # to compute z-scores
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
import sqlite3                                            # to interact with the database
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF

%matplotlib inline                                 
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns                                     # also improves the look of plots
sns.set()
plt.rcParams['figure.figsize'] = 10, 5                    # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1                 # to fix issue with seaborn box plots; needed after import seaborn

ModuleNotFoundError: No module named 'treeviz'

## Part 1a: Hypothesis selection

Our research question is as follows: What is the influence of the seasons on the air quality?
For this question we have made the following hypothesis:
    1. The concenctration of NOx is directly proportional to the wind speed.
    2.The effect on PM10 and pm2.5. A higher wind speed decreases the concentration of these particles.
    3. The concentration of NO decreases with the increase of temperature  

For each of these hypothesis we have a reason why we think it is worth to investigate the topic. They are as follows:
    1. NOx or nitrogen oxides can react to form smog or acid rain, which both are bad for the local enviroment and to the health of humans.
    2. PM particles, either PM_10 or PM_2.5 are so small they can damage the tissue in the lungs and cause lung inflammation.
    3. Temperature can have an influence on the pollution of nitrogen monoxide as low temperatures can lead to a layer of warm air forming above the ground trapping cold air and pollutants (including nitrogen monoxide) close to the ground.
Al these reasons were interesting to us since they all effect the health and reduce the air quality.


## Part 1b: Hypothesis refinement

The *concentration of NOx* decreases with *high wind speeds*.

1.
*concentration of NOx*
    
    what nitrogen oxides are included in the concentration of NOx( which ones will we measure)?

    how will we measure NOx?
        -what do we consider high/low?
        -what time periods will we choose?
    
    should we measure the different oxides differently/separately?
    
*high wind speeds*
   
    what do we consider high wind speeds?

    what do we consider low wind speeds?
    
    are there variations in wind speed we should consider?
    
    how will we treat extreme wind events?
       
    
confunding variables
   
    what other factors might affect NOx concentration?
        -temperature,humidity...?
        -are there any local stations, traffic... that might increase these values?
    
    


2. 
*concentration of NOx*
    The concentration of NOx is the dependent variable and refers to the concentration of Nitrogen Oxides (NOx) in 𝜇g/m^3
    we will measure NO_2 and NO
    These will be measured separately as they each might have a different effect on the air quality.
    we will consider high concentrations anything over the 90th percentile, and low anything below the 10th percentile. 
    'normal'concentrations will be considered as those in the IQR.

*high wind speeds*
    Wind speed, on the other hand, is this project's independent variable and it is defined the following way.The mean wind 
    speed in metres per second (m/s) 10 minutes before each measurement is taken.
        Wind speeds between 0 and 5 m/s are considered low wind speeds, while those between 5 and 15 m/s are considered medium 
        and those above 15 m/s are considered high wind speeds.    
    extreme wind events will be isolated and examined separately. 
        
        
cofunding variables
    Additionally, temperature is going to be considered as a confounding variable as it has a positive correlation with the 
    concentration of NOx.(_should we only measure above/below certain values? how do we mitigate this?_)
    another thing that we must consider is the traffic and industries close to the stations as higer traffic causes more no2 to 
    be produced. therefore we will only look at street stations as more NOx is produced there.




3.
Finally, in order to conduct the assignment data collected from three stations, Maastricht, Groningen and Rotterdam is going to be used. Furthermore, data from three other stations surrounding each one of the previous stations is going to be used to avoid possible biases or mistakes that might have happened at a particular weather station.(specify which stations, give names and reason behind) 




4.
• what data (including in which format) you need

    data of daily mean NOx concentrations data and daily mean wind speed will be used in order to compare how it changes each season. then we will go deeper and asses the daily windspeeds and NOx concentrations of one of the seasons/months in order to compare how they affect each other. This means data will be separated into type of station, season, month, and day.  

• where you will get this data from

    data will be obtained from the following stations, the stations will be limited to street stationsas traffic levels 
    influence the amount of NOx produced.(specify which stations, give names) 

• which technologies you intend to use
    .......






The original hypothesis was “The concentration of NOx decreases with high wind speeds.”
We have refined it as follows:
    air quality
        during 2021(or a certain amount of years, decide as group)- to make the population more homogenuos, eliminating cofunding variables such as covid year.

        from urban areas with traffic and industrial activities- as we are only using data from street stations 

        considering different stations across the netherlands- as we only have data available from the netherlands

        shows that the concentration of different nitrogen oxides (such as NO_2 and NO)- to specify what oxides we will be assesing
        
        decrease with high wind speeds.- to have a comparison with another variable



therefore, the refined hypothesis becomes:
"air quality data during 2021 (*choose year with best results, and add extra info above*) from urban areas with traffic and industrial activities, considering different stations across the netherlands, shows that the concentration of different nitrogen oxides (such as NO_2 and NO) decrease with high wind speeds."



## Part 2: Queries and data cleaning

In the following all the coordinations of the datapoints will be mentioned for easy use.

 - Main point maastricht: lat: 50.906000, lon: 5.762000
 - Main point rotterdam: lat: 51.962000, lon 4.447000
 - Main point Schiphol: lat: 52.318000, lon: 4.790000

Smaller points close to maastricht:
- maastricht a2 nassaulaan: lat: 50.845941 lon  5.714745
- maastricht hoge fronten: lat: 50.852049  lon: 5.675796
- Maastricht-A2 Kasteel Hillenraadweg  50.859810   5.713810 

Smaller points close to rotterdam:
- Hoek van Holland-Berghaven  51.977628   4.121226
-  Rotterdam Zuid-Zwartewaalstraat  51.893617   4.487528
- Vlaardingen-Riouwlaan  51.914883   4.329430

Smaller points close to Schiphol:
- Hoofddorp-Hoofdweg  52.327464   4.715008
- Haarlem-Schipholweg  52.370508   4.642319  
- Badhoevedorp-Sloterweg  52.334003   4.774006



In [ ]:
# first assign the coordinates to different variables
lat_maastricht = '50.906000'
lon_maastricht = '5.762000'

lat_rotterdam = '51.962000'
lon_rotterdam = '4.447000'

lat_schiphol = '52.318000'
lon_schiphol = '4.790000'
# first point of maastricht is called m1 etc and rotterdam r1 etc and schiphol s1 etc

lat_m1 = '50.845941'
lon_m1 = '5.714745'

lat_m2 = '50.852049'
lon_m2 = '5.675796'

lat_m3 = '50.859810'
lon_m3 = '5.713810'

lat_r1 = '51.977628'
lon_r1 = '4.121226' 

lat_r2 = '51.893617' 
lon_r2 = '4.487528'

lat_r3 = '51.914883'
lon_r3 = '4.329430' 

lat_s1 = '52.327464'
lon_s1 = '4.715008'

lat_s2 = '52.370508'
lon_s2 = '4.642319'

lat_s3 = '52.334003'
lon_s3 = '4.774006'

compound= 'no_x' #this assigns the particle we want to investigate to the variable compound
weather_factor = 'wind_speed'

In [ ]:
# need to connect to the correct database
conn = sqlite3.connect('./datasets/aqw.db')

In [ ]:
# now that all the coordinates are assigned to variables it will be easier to make the query's per point
# let's start with the three main points
#MSTRCHT = maastricht
QUERY_MSTRCHT1 = """
SELECT *
FROM weather_stations
WHERE latitude = '50.906000'  AND longitude = '5.762000'

"""

df_mstrcht1 = pd.read_sql_query(QUERY_MSTRCHT1, conn)
df_mstrcht1
# this code gives a code which we can use



In [ ]:
QUERY_RTRDM1 = """
SELECT *
FROM weather_stations
WHERE latitude = '51.962000' AND longitude = '4.447000'

"""
df_rtrdm1 = pd.read_sql_query(QUERY_RTRDM1, conn)
df_rtrdm1

In [ ]:
QUERY_SCHIPHOL1 = """
SELECT *
FROM weather_stations
WHERE latitude = '52.318000' AND longitude = '4.790000'

"""

df_sch1 = pd.read_sql_query(QUERY_SCHIPHOL1, conn)
df_sch1

In [ ]:
# now that we have all the codes for the correct main points it will be neccesary to also do the same for the smaller points
# However these points are in the database called air_quality_stations

query_m1 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '50.845941' AND longitude = '5.714745'

"""
df_m1 = pd.read_sql_query(query_m1, conn)
df_m1
# this gives the correct code for this station
# the same will be done for all the other smaller stations

In [ ]:
query_m2 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '50.852049' AND longitude = '5.675796'

"""
df_m2 = pd.read_sql_query(query_m2, conn)
df_m2

In [ ]:
query_m3 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '50.859810' AND longitude = '5.713810'

"""
df_m3 = pd.read_sql_query(query_m3, conn)
df_m3

In [ ]:
# now all the stations for main point rotterdam
query_r1 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '51.977628' AND longitude = '4.121226'
"""
df_r1 = pd.read_sql_query(query_r1, conn)
df_r1

In [ ]:
query_r2 = """
SELECT *
FROM air_quality_stations
WHERE latitude =  '51.893617'  AND longitude = '4.487528'
"""
df_r2 = pd.read_sql_query(query_r2, conn)
df_r2

In [ ]:
query_r3 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '51.914883' AND longitude = '4.329430' 
"""
df_r3 = pd.read_sql_query(query_r3, conn)
df_r3

In [ ]:
query_s1 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '52.327464' AND longitude = '4.715008'
"""
df_s1 = pd.read_sql_query(query_s1, conn)
df_s1

In [ ]:
query_s2 = """
SELECT *
FROM air_quality_stations
WHERE latitude = '52.370508' AND longitude =  '4.642319'
"""
df_s2 = pd.read_sql_query(query_s2, conn)
df_s2

In [ ]:
query_s3 = """
SELECT * 
FROM air_quality_stations
WHERE latitude = '52.334003' AND longitude = '4.774006'
"""
df_s3 = pd.read_sql_query(query_s3, conn)
df_s3
# now that we have all the codes for the stations we can start gathering the data from each of the stations and start plotting them

In [ ]:
#first redifine the weather factors
#this part is just figuring out how to get no_x in the dataset with the wind speed etc like they did in ga1 



In [ ]:
# let's start with creating databases for the main points
# then get a representation of the data for this point using a query
# I've only selected the wind speed for now 
query_mstrcht_data = """

SELECT *
FROM weather_data
WHERE station_code = '380'
"""

df_mstrcht_data = pd.read_sql_query(query_mstrcht_data, conn)
df_mstrcht_data

In [ ]:
# dataset of rotterdam station
# the weather factors don't have to be defined again since this proceeds on the previous code

query_rtrdm_data = """
SELECT wind_speed
FROM weather_data
WHERE station_code = 344
"""
df_rtrdm_data = pd.read_sql_query(query_rtrdm_data, conn)
df_rtrdm_data

In [ ]:
query_schiphol_data = """
SELECT *
FROM weather_data
WHERE station_code = 240
"""
df_schiphol_data = pd.read_sql_query(query_schiphol_data, conn)
df_schiphol_data


In [ ]:
# try to merge these three dataframes togeter
df_two = pd.merge(df_mstrcht_data, df_rtrdm_data, on='datetime' )
df_two




## Part 3. Hypothesis testing and interpretation

In [ ]:
# Use this cell as you like, and add more cells as needed.

## Part 4. Polishing and pitching results

In [ ]:
# This section is only for generating figures if you need it. You may leave it empty.

# Feedback

Please fill in this questionaire to help us improve this course for the next year. Your feedback will be anonymized and will not affect your grade in any way!

### How many hours did you spend on these exercises?

Assign a number to `feedback_time`.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_1] (0 point)

#// END_FEEDBACK [Feedback_1] (0 point)

import numbers

assert isinstance(feedback_time, numbers.Number), "Please assign a number to feedback_time"
print(feedback_time)

### How difficult did you find these exercises?

Assign an integer to `feedback_difficulty`, on a scale 0 - 10, with 0 being very easy, 5 being just right, and 10 being very difficult.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_2] (0 point)

#// END_FEEDBACK [Feedback_2] (0 point)

import numbers

assert isinstance(feedback_difficulty, numbers.Number), "Please assign a number to feedback_difficulty"
print(feedback_difficulty)

### (Optional) What did you like?

Assign a string to `feedback_like`.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_3] (0 point)

#// END_FEEDBACK [Feedback_3] (0 point)

### (Optional) What can be improved?

Assign a string to `feedback_improve`. Please be specific, so that we can act on your feedback. For example, mention the specific exercises and what was unclear.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_4] (0 point)

#// END_FEEDBACK [Feedback_4] (0 point)




## How to Submit Your Work

1. **Before submitting**, you must run your notebook by doing **Kernel > Restart & Run All**.  
   Make sure that your notebook runs without errors **in linear order**.
1. Remember to rename the notebook as explained at the beginning of this notebook.
1. Submit the executed notebook with your work
   for the appropriate assignment in **Canvas**.
1. In the **Momotor** tab in Canvas,
  you can select that assignment again to find some feedback on your submitted work.
  If there are any problems reported by _Momotor_,
  then you need to fix those,
  and **resubmit the fixed notebook**.

In case of a high workload on our server
(because many students submit close to the deadline),
it may take longer to receive the feedback.




---

In [ ]:
# List all defined names
%whos

---

# (End of Notebook) <span class="tocSkip"></span>

&copy; 2017-2023 - **TU/e** - Eindhoven University of Technology